# Progress Plotting

This notebook plots the pre-computed aggregate data. It's a stand-in for the future web app without the Athena bits of the query prototype notebook.

In [ ]:
import datetime
import math

import boto3
import matplotlib.pyplot as plt
import pandas as pd
import pytz

In [ ]:
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

In [ ]:
wheel_diameter = 8.5 # inches, not quite the 9" advertised, I measured
wheel_circumference = math.pi * wheel_diameter / 12 / 5280 # miles

In [ ]:
s3 = boto3.client('s3')

In [ ]:
tz = pytz.timezone('America/New_York')

In [ ]:
resp = s3.get_object(Bucket='honey-data-public', Key='prior-7-day-window.csv')
prior_df = pd.read_csv(resp['Body'])

In [ ]:
try:
    offset = prior_df.iloc[0].iloc[0]
except:
    offset = 0

In [ ]:
resp = s3.get_object(Bucket='honey-data-public', Key='7-day-window.csv')
week_df = pd.read_csv(resp['Body'])

In [ ]:
week_df['datetime_utc'] = pd.to_datetime(week_df.datetime_hour)
week_df['datetime'] = week_df.datetime_utc.dt.tz_convert(tz)
week_df.set_index('datetime', inplace=True)

In [ ]:
cumsum_df = week_df[['cumsum_rotations']] + offset
#cumsum_df = cumsum_df.reindex(pd.date_range(week_df.index.min(), week_df.index.max(), freq='1h'), method='ffill')

In [ ]:
cumsum_df.index.max() - cumsum_df.index.min()

In [ ]:
_, ax = plt.subplots(figsize=(15, 5))
(cumsum_df * wheel_circumference).rename(columns={'cumsum_rotations': 'miles'}).plot(ax=ax)
ax.set_facecolor('white')

In [ ]:
now = datetime.datetime.utcnow()
now = now.replace(hour=12, minute=0, second=0, microsecond=0)
yesterday = now - datetime.timedelta(days=1)

In [ ]:
week_df.loc[str(yesterday):str(now)].sum_rotations.sum() * wheel_circumference